In [1]:
import argparse
import os

import numpy as np
import torch
import scipy
from scipy.sparse import coo_matrix
import scipy.sparse as sp
from torch_geometric.data import Data

C:\Users\soha9\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def check_generate_random(A, u, b):
    tmp = A.toarray()
    print('A shape: ', A.shape)
    print('b shape: ', b.shape)
    print('x shape: ', u.shape)
    residual_ = np.linalg.norm(b.reshape(-1,1) - A @ u.reshape(-1,1))
    print('residual for 1 generate_sparse_random: ', residual_)

In [3]:
def generate_sparse_random(n, alpha=1e-4, random_state=0, sol=False, sym=0, check=True):
    # We add to spd matricies since the sparsity is only enforced on the cholesky decomposition
    # generare a lower trinagular matrix
    # Random state
    rng = np.random.RandomState(random_state)
    
    if n == 100_000:
        zero_prob = rng.uniform(0.999, 0.9998)
    elif n > 5000 and n <= 10_000:
        zero_prob = rng.uniform(0.995, 0.998)
    elif n >= 1000 and n <= 5_000:
        zero_prob = rng.uniform(0.993, 0.9965)
    elif n == 1_000 or n == 2_000:
        zero_prob = rng.uniform(0.98, 0.999)
    elif n == 100:
        zero_prob = rng.uniform(0.96, 0.99)
    elif n <= 50:
        zero_prob = 0.9
    else:
        raise NotImplementedError(f"Can\'t generate sparse matrix for n={n}")
    
    # old code:
    # S = rng.binomial(1, (1 - zero_prob), size=(n, n))
    # M = rng.normal(0, 1, size=(n, n))
    # M = S * M # enforce sparsity
    
    nnz = int((1 - zero_prob) * n ** 2)
    rows = [rng.randint(0, n) for _ in range(nnz)]
    cols = [rng.randint(0, n) for _ in range(nnz)]
    
    uniques = set(zip(rows, cols))
    rows, cols = zip(*uniques)
    
    # generate values
    vals = np.array([rng.normal(0, 1) for _ in cols])
    mean = np.mean(vals)
    std = np.std(vals)
    vals = (vals - mean) / std

    M = coo_matrix((vals, (rows, cols)), shape=(n, n))
    I = scipy.sparse.identity(n)
    
    if sym:
        # create spd matrix
        A = M @ M.T + alpha * I    
    else:
        A = M + alpha * I
        
    b = rng.uniform(0, 1, size=n)
    
    if check:
        x = scipy.sparse.linalg.spsolve(A, b)
        check_generate_random(A, x, b)

    else:
        if sol:
            x = scipy.sparse.linalg.spsolve(A, b)
            return A, x, b
        else:
            x = None
    
    return A, x, b

In [4]:
def gmres_without_preconditioner(A, b, u_true, tol, plot, method, path):

    n = A.shape[0]
    A = A.to_dense().numpy()
    b = b.to_dense().numpy()
    u_true = u_true.to_dense().numpy()
    error = A @ u_true
    print('error |x_true - x_hat|: ',error)
    
    global iteration
    iteration = 0
    residuals = []
    
    def callback(residual):
        global iteration
        iteration = iteration +1
        residual_norm = np.linalg.norm(residual)
        print(f'Iteration: {iteration} ==========> Residual: {residual_norm}')
        residuals.append(residual_norm)
            
    u_gmres, info = gmres(A, b, tol=tol, callback=callback, maxiter=n)
    u_gmres = u_gmres.reshape(-1,1)
    u_true = u_true.reshape(-1,1)

    error = np.linalg.norm(u_gmres - u_true)
    print('error |x_true - x_hat|: ',error)
    iterations_ = iteration
    
    if plot:
        print(f"Number of iterations for {method}:", iteration)
        plt.figure(1)
        plt.plot(residuals, label=method)
        plt.title(f'GMRES for random non-symmetric data (n={n})')
        plt.xlabel('# iteration')
        plt.ylabel('residual error')
        plt.yscale('log')
        plt.legend()
        plt.savefig(f'{path}/{method}_gmres.png')
    
    return iterations_, residuals

In [5]:
A, x, b = generate_sparse_random(n=1000, sol=True, sym=0)

A shape:  (1000, 1000)
b shape:  (1000,)
x shape:  (1000,)
residual for 1 generate_sparse_random:  0.0028839910232729363
